In [3]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
import numpy as np
from sklearn.externals import joblib
import matplotlib.pyplot as plt
import itertools
import re
import nltk

In [4]:
true = pd.read_csv("True.csv")

In [5]:
true.head(3)

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"


In [6]:
fake = pd.read_csv("Fake.csv")

In [7]:
fake.head(3)

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"


In [8]:
fake.shape

(23481, 4)

In [9]:
true.shape

(21417, 4)

In [10]:
true['fact'] = 1

In [11]:
fake['fact'] = 0

In [12]:
fake.head(3)

,title,text,subject,date,fact
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0


In [13]:
true.head(3)

,title,text,subject,date,fact
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1


In [14]:
dataTotal = [true.loc[0:5000][:], fake.loc[0:5000][:]]

In [15]:
totalData = pd.concat(dataTotal)

In [16]:
totalData.shape

(10002, 5)

In [17]:
totalData.sample(frac=1).reset_index(drop=True)

,title,text,subject,date,fact
0,Colbert PERFECTLY Explains The Horror Of A Wo...,Donald Trump has always shown himself to be im...,News,"December 2, 2016",0
1,This ‘Duck Dynasty’ Star Wants To Baptize Don...,"When thinking about this election, it s probab...",News,"September 12, 2016",0
2,Government Ethics Office Systems Crash After ...,Kellyanne Conway s decision to provide Ivanka ...,News,"February 9, 2017",0
3,Pence Says There Is ‘Too Much Talk’ About Rac...,"While Charlotte, North Carolina faces a state ...",News,"September 22, 2016",0
4,Lebanon's Hezbollah calls U.S. strike on Syria...,BEIRUT (Reuters) - Lebanese Shi’ite group Hezb...,politicsNews,"April 7, 2017",1
...,...,...,...,...,...
9997,Senator McCain says he will offer Afghanistan ...,WASHINGTON (Reuters) - Republican U.S. Senator...,politicsNews,"July 31, 2017",1
9998,Michigan state House approves carrying conceal...,(Reuters) - Legislation to allow Michigan gun ...,politicsNews,"June 8, 2017",1
9999,Trump invites Republican senators to healthcar...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"July 18, 2017",1
10000,"Trump to limit Cuba travel, restrict business ...",WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"June 15, 2017",1


In [18]:
xAxis = totalData.drop("fact",axis=1)

In [19]:
yAxis = totalData['fact']

In [20]:
totalData = totalData.dropna()

In [21]:
stringTotal = totalData.copy()

In [22]:
stringTotal.reset_index(inplace = True)

In [23]:
stringTotal.head()

,index,title,text,subject,date,fact
0,0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [24]:
stringTotal['text'][34]

' KING OF PRUSSIA, Pennsylvania/WASHINGTON (Reuters) - In the Fox & Hound sports bar, next to a shopping mall in suburban Philadelphia, four Democrats are giving speeches to potential voters as they begin their journey to try to unseat Republican congressman Pat Meehan in next year’s elections. Winning this congressional district - Pennsylvania’s 7th - is key to Democrats’ hopes of gaining the 24 seats they need to retake the U.S. House of Representatives next November. The stakes are high - control of the House would allow them to block President Donald Trump’s legislative agenda. On the surface, Democrats face a significant hurdle. In nearly two-thirds of 34 Republican-held districts that are top of the party’s target list, household income or job growth, and often both, have risen faster than state and national averages over the past two years, according to a Reuters analysis of census data.\xa0(Graphic:\xa0tmsnrt.rs/2Bgq29K) That is potentially vote-winning news for Republican incu

In [25]:
stringTotal['fact'][34]

1

# **Natural Language Processing**
This step makes program more efficient and lessens the contrains on the machine learning aspect of the program

In [27]:
singleStem = PorterStemmer()

In [ ]:
nltk.download('stopwords')
limit = []
for i in range(0,len(stringTotal)):
  review = re.sub('[^a-zA-Z]', ' ', stringTotal['text'][i])
  review = review.lower()
  review = review.split()
  review = [singleStem.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  limit.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
vector = TfidfVectorizer(max_features=5000, ngram_range=(1,3))

In [ ]:
vector.get_params()

In [ ]:
xAxis = vector.fit_transform(limit).toarray()

In [ ]:
yAxis = stringTotal['fact']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(xAxis, yAxis, test_size=0.2, random_state=0)

In [ ]:
data_count = pd.DataFrame(X_train, columns=vector.get_feature_names())

In [ ]:
data_count.head(3)

In [ ]:
identifier = PassiveAggressiveClassifier(max_iter=1000)

In [ ]:
identifier.fit(X_train, y_train)

In [ ]:
predictor = identifier.predict(X_test)
accuracy = metrics.accuracy_score(y_test,predictor)

In [ ]:
print(accuracy)

In [ ]:
review = re.sub('[^a-zA-Z]', ' ', true['text'][16888])
review = review.lower()
review = review.split()
review = [singleStem.stem(word) for word in review if not word in stopwords.words('english')]
review = ' '.join(review)


In [ ]:
inputData = vector.transform([review]).toarray()

In [ ]:
pd.DataFrame(inputData, columns = vector.get_feature_names())

In [ ]:
identifier.predict(inputData)

In [ ]:
joblib.dump(identifier, 'model.pkl')

In [ ]:
joblib.dump(vector, 'tfidfvect.pkl')